In [1]:
#IMPORTS
#Install+Import Packages
!pip install mechanize
import random
import re
import mechanize
import numpy as np
from random import *
import matplotlib.pyplot as plt
!pip install Bio
from Bio import SeqIO
from Bio.Seq import Seq
from collections import Counter 
from Bio.Cluster import kcluster
import statistics
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import difflib
import seaborn as sns; sns.set_theme()
from collections import defaultdict
import collections
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
from collections import OrderedDict
import matplotlib.pyplot as plt
#!pip install ipython-autotime

In [2]:
#Upload Data and Define Parameters

file_type = '.fastq'

aa_sequences = open('/Users/zoeweiss/Desktop/Desktop/lab(s)/szostak/spring 2022/aminoacylation/Str_S1_L001_R2_001.fastq').readlines()
rna_only_sequences = open('/Users/zoeweiss/Desktop/Desktop/lab(s)/szostak/spring 2022/aminoacylation/Str-NaOH_S2_L001_R2_001.fastq').readlines()

#Take the reverse complement? Enter 'yes' or 'no'
rc = 'yes'

#Quality Cutoff - what fraction of positions should have <1% error probability?
quality_cutoff = 0.9


#Position motif1 starts at
motif_start = 1
#Motif1 define
motif = 'AAA'
#Position motif2 starts at
motif2_start = 23
#Motif2 define
motif2_rev = 'TTTGATCGTCGGACTGTAGAACTCTGAAC'



In [3]:
def preprocess_seqs(list_of_seqs):

    if file_type == '.fastq':
        seqs = list_of_seqs[1::4]
        qualities = list_of_seqs[3::4]
        print('Total seqs: '+str(len(seqs)))

        #Filter by quality
        high_quality = []
        acceptable_quals_1_percent_error = [5,6,7,8,9,':',';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
        for seq in range(len(qualities)):
            overlap=0
            for char in list(set(qualities[seq]).intersection(acceptable_quals_1_percent_error)):
                overlap+=qualities[seq].count(char)
            if overlap/len(qualities[1])>quality_cutoff:
                high_quality.append(seqs[seq].strip("\n"))
        print('High quality seqs: '+str(len(high_quality)))
    
    if file_type == '.fasta':
        high_quality = list_of_seqs[1::2]
        print('Total seqs: '+str(len(high_quality)))
    
    #Trim to keep only seqs with constant region
    trimmed = []
    for seq in high_quality:
        if motif2_rev in seq:
            if len(seq[seq.index(motif2_rev)-40:seq.index(motif2_rev)])>0:
                trimmed.append(seq[seq.index(motif2_rev)-40:seq.index(motif2_rev)])
                
    print('Seqs after trimming: '+str(len(trimmed))) 
    
    #Take RC if desired
    pre_trim = []
    if rc == 'yes':
        rc_seqs = []
        for sequence in trimmed:
            rc_seqs.append(str(Seq(sequence).reverse_complement()).strip('\n'))
        pre_trim = rc_seqs
    else:
        for sequence in trimmed:
            pre_trim.append(str(Seq(sequence)).strip('\n'))
    if rc == 'yes':
        print('Reverse Comp = '+str(rc)) 
    
    #All processed seqs
    processed_seqs = pre_trim
    #Unique seqs
    processed_seqs_unique = list(set(pre_trim))
    
    return processed_seqs, processed_seqs_unique
    

#Function to get secondary structure on RNA Fold
def get_energy_ss(seq):
  url = "http://rna.tbi.univie.ac.at//cgi-bin/RNAWebSuite/RNAfold.cgi"
  br = mechanize.Browser()
  br.set_handle_robots(False) # ignore robots
  br.open(url)
  br.select_form(name="form")
  br["SCREEN"] = seq
  res = br.submit()
  content = res.read()
  energy = int(re.sub("[^0-9]", "", str(content).split('thermodynamic ensemble is')[1].split('kcal/mol')[0]))
  structure = str(content).split('STRUCTURE=')[1].split('target')[0].split('"')[0]
  return energy, structure



In [4]:
#AA CONTROL
#STATISTICS (note: this takes a few mins per round)
aa_ctrl_processed_seqs, aa_ctrl_processed_seqs_unique = preprocess_seqs(aa_sequences)
aa_ctrl_top_seqs = Counter(aa_ctrl_processed_seqs).most_common(100)
print(aa_ctrl_top_seqs[:10])

Total seqs: 3006527
High quality seqs: 2833045
Seqs after trimming: 1836981
Reverse Comp = yes
[('ACGGGTCGATTTATAGGGTAATGGGAGCGATCGATGCGTG', 2), ('TGTTACTATTTGTTTCAGCTGAGGGTAGTATCGCAGGTTA', 2), ('GTAGCATGAACAGGTGTAGGCTGCGCATCTGTGGTATGCA', 2), ('GGAGTAGGTAACAGTTGGCATAAGATTGAGGTCGGTGGTG', 2), ('GCCGGCGGTTTTATTAATCTTTGTTATGCTGCAATAGGGG', 2), ('CAGCGATCATGGTTGACATGGAGGACATGGTAGGTAGTAG', 2), ('GTTTTTTTCACCAGGATGACAGCGCGCGGGGAGCATGGAC', 2), ('AGGAGTTACATGGCGATAAGGTATGTAACATTTTTTGTAG', 2), ('GTTAGCCGCCTATATATAGGGAGAGGCAAGCGCCAGTGAA', 2), ('TGTGTTGGTTATACCAAAGCAGAAGTGCATTTGAGATTGG', 2)]


In [5]:
#RNA CONTROL
#STATISTICS (note: this takes a few mins per round)
rna_ctrl_processed_seqs, rna_ctrl_processed_seqs_unique = preprocess_seqs(rna_only_sequences)
rna_ctrl_top_seqs = Counter(rna_ctrl_processed_seqs).most_common(100)
print(rna_ctrl_top_seqs[:10])



Total seqs: 3449734
High quality seqs: 3166298
Seqs after trimming: 1315976
Reverse Comp = yes
[('AGTGAGCCAGATAAGCCCCGCTGTTGGAGGGTGTAATATA', 2), ('TAGGGAAAGTACAACTGCACTGAGCTGGTTGGGATTAGGG', 2), ('TACAGTGGCGATTCTGCGCACTGGGGCGTTGGCTGGCTAA', 2), ('GGGGGCAAGAAGTTTGGTCGGATTGTAGGCGGAGACGTTC', 2), ('GGCATCTCAATAAAGTACTTACCGTGGGCGACAGCGGGCA', 2), ('AGAGGCGATCGGAGGGTCACTTTGTAAGTCGCGTGGGGGG', 2), ('CTTTGTTTCGATGATGATGCGTATGGTTTATCATAAGGGC', 2), ('AATGGGGTATGCGTATATCATTAGAGGGTAGGCATTGAGG', 2), ('TGAGCAATTTGGGGGAAGAGATTATATACTTATTGAGAAA', 2), ('AGGGCAGGGTCGTGCCAATGGGTGGTCCGGGCGGAATAGC', 2)]


In [6]:
#AA NOT RNA 
#STATISTICS (note: this takes a few mins per round)

aa_rna_overlap = list(set(aa_ctrl_processed_seqs) & set(rna_ctrl_processed_seqs))

aa_not_rna_seqs = aa_ctrl_processed_seqs
counter = 0
for seq in range(len(aa_rna_overlap)):
    counter+=1
    if counter%100==0:
        print(str(counter)+'/'+str(len(aa_rna_overlap)))
        
    aa_not_rna_seqs.remove(aa_rna_overlap[seq])

print('Seqs in AA but not RNA: '+str(round(len(aa_not_rna_seqs))))

aa_not_rna_top_seqs = Counter(aa_not_rna_seqs).most_common(100)
print(aa_not_rna_top_seqs[:10])


Seqs in AA but not RNA: 1836981
[('ACGGGTCGATTTATAGGGTAATGGGAGCGATCGATGCGTG', 2), ('TGTTACTATTTGTTTCAGCTGAGGGTAGTATCGCAGGTTA', 2), ('GTAGCATGAACAGGTGTAGGCTGCGCATCTGTGGTATGCA', 2), ('GGAGTAGGTAACAGTTGGCATAAGATTGAGGTCGGTGGTG', 2), ('GCCGGCGGTTTTATTAATCTTTGTTATGCTGCAATAGGGG', 2), ('CAGCGATCATGGTTGACATGGAGGACATGGTAGGTAGTAG', 2), ('GTTTTTTTCACCAGGATGACAGCGCGCGGGGAGCATGGAC', 2), ('AGGAGTTACATGGCGATAAGGTATGTAACATTTTTTGTAG', 2), ('GTTAGCCGCCTATATATAGGGAGAGGCAAGCGCCAGTGAA', 2), ('TGTGTTGGTTATACCAAAGCAGAAGTGCATTTGAGATTGG', 2)]


In [7]:
#Get the SS of the top sequences
ss_of_top_seqs=[]
for seq in aa_not_rna_top_seqs[:2]:
  if aa_not_rna_top_seqs.index(seq)%2==0:
    print(aa_not_rna_top_seqs.index(seq))
  ss_of_top_seqs.append(get_energy_ss(seq[0])[1])

0


In [8]:
#Cluster by secondary structure
k = 20
matrix = np.asarray([np.fromstring(str(s), dtype=np.uint8) for s in ss_of_top_seqs]);
kmeans = KMeans(init="random", n_clusters=k,n_init=10,max_iter=300,random_state=42)
kmeans.fit(matrix)
labels = kmeans.labels_[:]

all_clusters = []
for cluster_num in range(0, k):
  seqs_in_cluster = []
  for seq in range(len(labels)):
    if labels[seq] == cluster_num:
      seqs_in_cluster.append(ss_of_top_seqs[seq])
  all_clusters.append(seqs_in_cluster)

labels

order_of_clusters = []
for rank in labels:
    if rank not in order_of_clusters:
        order_of_clusters.append(rank)

for cluster_num in order_of_clusters:
    print(all_clusters[cluster_num])
        

<ipython-input-8-429886209a86>:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  matrix = np.asarray([np.fromstring(str(s), dtype=np.uint8) for s in ss_of_top_seqs]);


ValueError: n_samples=2 should be >= n_clusters=20.

In [16]:
counts = {k: v for k, v in sorted(dict(Counter(aa_not_rna_seqs)).items(), key=lambda item: item[1], reverse= True)}
list(counts.values()).count(1), list(counts.values()).count(2)


(1835369, 806)